In [1]:
from collections import OrderedDict
import time
import logging, warnings

from qiskit.optimization.algorithms import CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from random_lp.lp_random_gen import create_models
from utilities.helpers import create_qaoa_meo

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [3]:
DIR = 'TEST_DATA' + "/" + time.strftime("%d_%m_%Y")
DIR

'TEST_DATA/28_03_2021'

In [4]:
qps = create_models([DIR + "/DENSE/", DIR + "/SPARSE/"])
print(qps.keys())

odict_keys(['test_dense_3', 'test_dense_5', 'test_dense_7', 'test_sparse_2', 'test_dense_9', 'test_sparse_3', 'test_dense_11', 'test_sparse_4', 'test_dense_13', 'test_sparse_5', 'test_dense_15', 'test_sparse_6'])


In [5]:
cplex = CplexOptimizer()
qaoa = create_qaoa_meo()

In [6]:
max_qubits = qaoa.min_eigen_solver.quantum_instance.backend.configuration().to_dict()['n_qubits']
max_qubits

24

In [7]:
results = OrderedDict()
for qp_name, qp in qps.items() :
    print(qp_name)
    print("number of qubits: ", qp.qubo.get_num_vars())
    if qp.qubo.get_num_vars() > max_qubits:
        print("Too many qubits!")
        break
        
    
    res = qaoa.solve(qp)
    results[qp_name] = res
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = cplex.solve(qp)
        if res.fval != cres.fval:
            print("\n optimal value QAOA   "+str(res.fval) + "  ,  cplex:"+ str(cres.fval))
    else:
        print(res.status)

test_dense_3
number of qubits:  4
success
test_dense_5
number of qubits:  10
success
test_dense_7
number of qubits:  12
success

 optimal value QAOA   2.0  ,  cplex:-5.0
test_sparse_2
number of qubits:  13
success
test_dense_9
number of qubits:  17
success

 optimal value QAOA   -1.0  ,  cplex:-4.0
test_sparse_3
number of qubits:  17
success

 optimal value QAOA   -1.0  ,  cplex:-5.0
test_dense_11
number of qubits:  25
Too many qubits!
